<a href="https://colab.research.google.com/github/Kensheretohelp/were-here-to-help-podcast-analytics/blob/main/TRIALheretohelpvfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 pandas tqdm openai


In [4]:
import requests
import re
import json
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from openai import OpenAI

# ==========================================
# 0. CONFIG
# ==========================================

# 🔑 Put your API key here OR rely on env var OPENAI_API_KEY
client = OpenAI(api_key="xxxxxxx")   # or just OpenAI() if env var is set

BASE_LIST_URL = "https://podscripts.co/podcasts/were-here-to-help"
HEADERS = {"User-Agent": "Mozilla/5.0 (KenPodcastScraper/1.0)"}

# to start, keep it 1–20 until you like the results
EPISODE_MIN = 1
EPISODE_MAX = 235

REQUEST_SLEEP = 4  # seconds between HTTP page hits


# ==========================================
# 1. EPISODE LIST SCRAPER
# ==========================================
def get_episode_list():
    episodes = []
    page = 1

    while True:
        url = f"{BASE_LIST_URL}?page={page}"
        print(f"Scraping list page {page}: {url}")
        try:
            resp = requests.get(url, headers=HEADERS, timeout=20)
        except Exception as e:
            print(f"  !! Error on list page {page}: {e}")
            break

        if resp.status_code != 200:
            print(f"  !! Status {resp.status_code}, stopping list scrape.")
            break

        soup = BeautifulSoup(resp.text, "html.parser")
        links = soup.select("a[href*='/podcasts/were-here-to-help/']")
        found = 0

        for a in links:
            href = a.get("href") or ""
            if "page=" in href:
                continue
            if "/podcasts/were-here-to-help/" not in href:
                continue

            m = re.search(r"/(\d{1,3})-", href)
            if not m:
                continue
            ep_num = int(m.group(1))

            full_url = "https://podscripts.co" + href
            text = a.get_text(strip=True)

            episodes.append({
                "episode_num": ep_num,
                "title_raw": text,
                "episode_url": full_url
            })
            found += 1

        if found == 0:
            break

        page += 1
        time.sleep(REQUEST_SLEEP)

    # dedupe + sort
    seen = set()
    unique = []
    for e in episodes:
        if e["episode_num"] not in seen:
            seen.add(e["episode_num"])
            unique.append(e)

    unique.sort(key=lambda x: x["episode_num"])
    return unique


# ==========================================
# 2. SAFE GET + TRANSCRIPT EXTRACTOR
# ==========================================
def safe_get(url, retries=5):
    for attempt in range(retries):
        try:
            r = requests.get(url, headers=HEADERS, timeout=20)
        except Exception as e:
            print(f"  !! Request error: {e}")
            time.sleep(2 + attempt)
            continue

        if r.status_code == 200:
            return r.text

        if r.status_code == 429:
            wait = 3 + attempt * 2
            print(f"  !! 429 rate-limit at {url}, waiting {wait}s")
            time.sleep(wait)
            continue

        print(f"  !! HTTP {r.status_code} at {url}")
        time.sleep(2)

    print(f"  !! Failed to fetch after retries: {url}")
    return None


def extract_transcript(html):
    """Prefer JSON transcription; fall back to HTML spans."""
    if not html:
        return None

    # Try JSON blob first
    m = re.search(r"window\.__INITIAL_STATE__\s*=\s*(\{.*?\})\s*;<",
                  html, re.DOTALL)
    if m:
        try:
            state_json = json.loads(m.group(1))
        except Exception:
            state_json = None

        transcription_url = None
        if state_json:
            # path A
            try:
                transcription_url = state_json["pageData"]["episode"]["transcriptionUrl"]
            except Exception:
                pass
            # path B
            if not transcription_url:
                try:
                    transcription_url = state_json["episode"]["transcriptionUrl"]
                except Exception:
                    transcription_url = None

        if transcription_url:
            if transcription_url.startswith("/"):
                transcription_url = "https://podscripts.co" + transcription_url

            tr = safe_get(transcription_url)
            if tr:
                try:
                    tr_json = json.loads(tr)
                    segments = None
                    if "transcription" in tr_json:
                        segments = tr_json["transcription"].get("segments")
                    if not segments and "segments" in tr_json:
                        segments = tr_json["segments"]

                    if segments:
                        parts = [seg.get("text", "").strip() for seg in segments]
                        full_text = " ".join(parts)
                        full_text = re.sub(r"\s+", " ", full_text).strip()
                        if len(full_text) > 100:
                            return full_text
                except Exception as e:
                    print("  !! Error parsing transcription JSON:", e)

    # Fallback: HTML text
    soup = BeautifulSoup(html, "html.parser")
    spans = soup.select("span.pod_text")
    if spans:
        txt = " ".join(s.get_text(" ", strip=True) for s in spans)
        txt = re.sub(r"\s+", " ", txt).strip()
        if len(txt) > 100:
            return txt

    return None


# ==========================================
# 3. GPT: EXTRACT CALLERS FROM FULL TRANSCRIPT
# ==========================================
def extract_callers_from_transcript(ep_num, transcript):
    """
    Send the whole transcript for ONE episode to GPT, ask it for
    a JSON object: { "callers": [ {caller_slot, caller_name, caller_location, reason_short}, ... ] }
    """
    prompt = f"""
You are analyzing a FULL transcript from the podcast "We're Here to Help".

Your job: extract ONLY real phone callers.

For each caller:
- They are clearly calling in (by phone).
- Hosts greet them or start "You're on We're Here to Help" / "Thanks for calling" etc.
- They give or confirm a first name.
- They say where they are calling from (city/region/state/country).
  - If location is NOT clear at all, set caller_location to null.
- They ask a real question or describe a real problem.

Ignore:
- Hosts (Jake, Gareth) talking.
- In-studio guests.
- Characters, bits, or sketches.
- Any non-caller voice.

Output a JSON object with this exact structure:

{{
  "callers": [
    {{
      "caller_slot": 1,
      "caller_name": "FirstName",
      "caller_location": "City, State/Country",
      "reason_short": "One short sentence describing why they called."
    }},
    ...
  ]
}}

Rules:
- caller_slot must be 1, 2, 3,... in order of first appearance in the episode.
- caller_name: first name only (no last names).
- caller_location: keep as spoken (e.g. "Bloomington, Indiana", "North County San Diego", "London, UK").
- If you're not sure about a caller, SKIP them (do not guess).
- If there are no callers, return {{"callers": []}}.

Here is the full transcript for episode {ep_num}:

\"\"\"{transcript}\"\"\"
"""

    # Use JSON mode for more reliable parsing
    for attempt in range(3):
        try:
            resp = client.chat.completions.create(
                model="gpt-4o-mini",
                response_format={"type": "json_object"},
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
            )
            content = resp.choices[0].message.content
            data = json.loads(content)
            callers = data.get("callers", [])
            # Basic sanity check
            if not isinstance(callers, list):
                callers = []
            return callers
        except Exception as e:
            print(f"  !! OpenAI error (attempt {attempt+1}): {e}")
            time.sleep(2 + attempt)

    print("  !! Failed to get callers for this episode.")
    return []
# ==========================================
# 3.5 TESTING
# ==========================================
def extract_call_segment(full_text, caller_obj, window=1200):
    """
    Extract only the transcript segment that contains THIS caller.
    Uses the caller's name to locate the call and returns a window around it.
    ~1200 characters before and after the caller's first appearance.
    """
    if not full_text or not caller_obj:
        return None

    name = caller_obj.get("caller_name")
    if not name:
        return None

    # Case-insensitive search for caller name
    i = full_text.lower().find(name.lower())
    if i == -1:
        # Fallback: try "I'm calling from" pattern
        m = re.search(r"calling from", full_text, re.IGNORECASE)
        if not m:
            return None
        i = m.start()

    start = max(0, i - window)
    end = min(len(full_text), i + window)

    seg = full_text[start:end].strip()
    seg = re.sub(r"\s+", " ", seg)
    return seg

# ==========================================
# 4. MAIN PIPELINE (FIXED)
# ==========================================
all_rows = []

episode_meta_all = get_episode_list()
print(f"\nTotal episodes found: {len(episode_meta_all)}")

episode_meta_range = [
    e for e in episode_meta_all
    if EPISODE_MIN <= e["episode_num"] <= EPISODE_MAX
]

print(f"Processing episodes {EPISODE_MIN}–{EPISODE_MAX}:",
      [e["episode_num"] for e in episode_meta_range])

for ep in tqdm(episode_meta_range, desc="Processing episodes"):
    ep_num = ep["episode_num"]
    ep_title = ep["title_raw"]
    ep_url = ep["episode_url"]

    print(f"\n=== Episode {ep_num}: {ep_title} ===")
    print("URL:", ep_url)

    html = safe_get(ep_url)
    if not html:
        print("  !! No HTML, skipping episode.")
        continue

    transcript = extract_transcript(html)
    if not transcript:
        print("  !! No transcript extracted.")
        continue

    print(f"  Transcript length: {len(transcript)} characters")

    callers = extract_callers_from_transcript(ep_num, transcript)
    print(f"  → Callers found: {len(callers)}")

    # Append one row per caller
    for c in callers:
        all_rows.append({
            "episode_num": ep_num,
            "episode_title_raw": ep_title,
            "episode_url": ep_url,
            "caller_slot": c.get("caller_slot"),
            "caller_name": c.get("caller_name"),
            "caller_location": c.get("caller_location"),
            "reason_short": c.get("reason_short"),
            "call_segment": extract_call_segment(transcript, c)
        })
if not all_rows[-1]["call_segment"]:
    print(f"  !! Warning: Could not extract call segment for caller {c.get('caller_name')} in episode {ep_num}")

# Save
df = pd.DataFrame(all_rows)
df.to_csv("podcast_callers_basic_1_20.csv", index=False)
print("\nSaved → podcast_callers_basic_1_20.csv")
print(df.head(20))


Scraping list page 1: https://podscripts.co/podcasts/were-here-to-help?page=1
Scraping list page 2: https://podscripts.co/podcasts/were-here-to-help?page=2
Scraping list page 3: https://podscripts.co/podcasts/were-here-to-help?page=3
Scraping list page 4: https://podscripts.co/podcasts/were-here-to-help?page=4
Scraping list page 5: https://podscripts.co/podcasts/were-here-to-help?page=5
Scraping list page 6: https://podscripts.co/podcasts/were-here-to-help?page=6
Scraping list page 7: https://podscripts.co/podcasts/were-here-to-help?page=7
Scraping list page 8: https://podscripts.co/podcasts/were-here-to-help?page=8
Scraping list page 9: https://podscripts.co/podcasts/were-here-to-help?page=9
Scraping list page 10: https://podscripts.co/podcasts/were-here-to-help?page=10
Scraping list page 11: https://podscripts.co/podcasts/were-here-to-help?page=11
Scraping list page 12: https://podscripts.co/podcasts/were-here-to-help?page=12

Total episodes found: 179
Processing episodes 1–235: [1, 

Processing episodes:   0%|          | 0/178 [00:00<?, ?it/s]


=== Episode 1: 0comments ===
URL: https://podscripts.co/podcasts/were-here-to-help/1-the-quest-of-wilhelmina
  Transcript length: 22868 characters


Processing episodes:   1%|          | 1/178 [00:02<06:20,  2.15s/it]

  → Callers found: 1

=== Episode 2: 0comments ===
URL: https://podscripts.co/podcasts/were-here-to-help/2-the-santa-dilemma
  Transcript length: 37061 characters


...

  → Callers found: 3

=== Episode 234: 0comments ===
URL: https://podscripts.co/podcasts/were-here-to-help/234-merlin-captain-ron-with-rory-scovel
  Transcript length: 70814 characters


Processing episodes:  99%|█████████▉| 177/178 [18:52<00:06,  6.98s/it]

  → Callers found: 2

=== Episode 235: 0comments ===
URL: https://podscripts.co/podcasts/were-here-to-help/235-rye-of-the-tiger-kevin-dont-like-deli-meats
  Transcript length: 54665 characters


Processing episodes: 100%|██████████| 178/178 [19:01<00:00,  6.41s/it]

  → Callers found: 4

Saved → podcast_callers_basic_1_20.csv
    episode_num episode_title_raw  \
0             1         0comments   
1             2         0comments   
2             2         0comments   
3             3         0comments   
4             3         0comments   
5             4         0comments   
6             5         0comments   
7             5         0comments   
8             6         0comments   
9             6         0comments   
10            7         0comments   
11            7         0comments   
12            8         0comments   
13            8         0comments   
14            9         0comments   
15            9         0comments   
16           10         0comments   
17           10         0comments   
18           12         0comments   
19           13         0comments   

                                          episode_url  caller_slot  \
0   https://podscripts.co/podcasts/were-here-to-he...            1   
1   https://podscript